In [2]:
from langchain_groq import ChatGroq

In [3]:

llm = ChatGroq(
  temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-44515?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FORM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return in JSON format containing the following keys:
    `role`,`experience`,`skills` and `description`.
    ### VALID JSON (NO PREAMBLE):
    """ 
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

Unfortunately, the provided text does not contain any job postings. It appears to be a list of departments, teams, and locations within the company, as well as links to various resources and policies.

However, I can provide you with a sample JSON output that matches the expected format, assuming we had some job postings to extract:

```json
[
  {
    "role": "Software Engineer",
    "experience": "3-5 years",
    "skills": ["Java", "Python", "Cloud Computing"],
    "description": "Design and develop scalable software applications"
  },
  {
    "role": "Data Analyst",
    "experience": "1-3 years",
    "skills": ["Excel", "SQL", "Data Visualization"],
    "description": "Analyze and interpret complex data sets to inform business decisions"
  },
  {
    "role": "Marketing Manager",
    "experience": "5-7 years",
    "skills": ["Marketing Strategy", "Brand Management", "Team Leadership"],
    "description": "Develop and execute marketing campaigns to drive business growth"
  }
]
```

If 

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': '3-5 years',
  'skills': ['Java', 'Python', 'Cloud Computing'],
  'description': 'Design and develop scalable software applications'},
 {'role': 'Data Analyst',
  'experience': '1-3 years',
  'skills': ['Excel', 'SQL', 'Data Visualization'],
  'description': 'Analyze and interpret complex data sets to inform business decisions'},
 {'role': 'Marketing Manager',
  'experience': '5-7 years',
  'skills': ['Marketing Strategy', 'Brand Management', 'Team Leadership'],
  'description': 'Develop and execute marketing campaigns to drive business growth'}]

In [19]:
!pip install pandas
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [16]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [17]:
links = collection.query(query_texts=["Experience in python","Expertise in React"],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [21]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [20]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Aman, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Aman, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Your Business Needs - AtliQ

Dear Hiring Manager,

I came across your job postings for a Software Engineer, Data Analyst, and Marketing Manager, and I'm excited to introduce you to AtliQ, an AI & Software Consulting company that can help you find the perfect candidates or even provide tailored solutions to support your business growth.

At AtliQ, we have a proven track record of empowering enterprises with scalable, process-optimized, cost-effective, and efficient solutions. Our team of experts has extensive experience in designing and developing software applications, analyzing complex data sets, and executing marketing campaigns to drive business growth.

Our capabilities align perfectly with your job requirements:

* For the Software Engineer role, our expertise in Java, Python, and Cloud Computing can help you design and develop scalable software applications.
* For the Data Analyst role, our proficiency in Excel, SQL, and Data Visualization can help y